**DATA preprocessing and create lable**

In [16]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as pyplot
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import string

In [17]:
DATA_PATH = os.path.join("DATA") # paht of the exported data
alphabet = string.digits


In [18]:
character,lables = [], []
alphabet = np.array(os.listdir(os.path.join(DATA_PATH)))
label_map = {lable:num for num,lable in enumerate(alphabet)}
for char in alphabet:
  for files in os.listdir(os.path.join(DATA_PATH,char)):
    res = np.load(os.path.join(DATA_PATH,char,files))
    character.append(res)
    lables.append(label_map[char])

X = np.array(character)
y = to_categorical(lables).astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.01)


In [19]:
with open ("label_map.txt","w") as file:
  for word in label_map.keys():
    file.write(f"{word}:{label_map[word]}\n")

In [20]:
X_train.shape

(5174, 63)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.callbacks import TensorBoard
import tensorflowjs as tfjs

In [22]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
model = Sequential()
model.add(Dense(64,activation="tanh",input_shape=(X_train.shape[1],)))
model.add(Dense(32,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dense(alphabet.shape[0],activation="softmax"))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
model.fit(X_train,y_train,epochs=200,callbacks=[tb_callback])

Epoch 1/200
162/162 [==============================] - 2s 9ms/step - loss: 2.0300 - categorical_accuracy: 0.2424
Epoch 2/200
162/162 [==============================] - 1s 4ms/step - loss: 1.3028 - categorical_accuracy: 0.5151
Epoch 3/200
162/162 [==============================] - 1s 5ms/step - loss: 0.6947 - categorical_accuracy: 0.7590
Epoch 4/200
162/162 [==============================] - 1s 4ms/step - loss: 0.4607 - categorical_accuracy: 0.8475
Epoch 5/200
162/162 [==============================] - 1s 4ms/step - loss: 0.3277 - categorical_accuracy: 0.8949
Epoch 6/200
162/162 [==============================] - 1s 4ms/step - loss: 0.2671 - categorical_accuracy: 0.9138
Epoch 7/200
162/162 [==============================] - 1s 4ms/step - loss: 0.2227 - categorical_accuracy: 0.9302
Epoch 8/200
162/162 [==============================] - 1s 3ms/step - loss: 0.2005 - categorical_accuracy: 0.9391
Epoch 9/200
162/162 [==============================] - 1s 3ms/step - loss: 0.1649 - categorical_

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 64)                4096      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_7 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_9 (Dense)              (None, 10)                650       
Total params: 9,994
Trainable params: 9,994
Non-trainable params: 0
____________________________________________________

In [25]:
model.save("number.h5")
tfjs.converters.save_keras_model(model, "number")

In [26]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()
multilabel_confusion_matrix(ytrue,yhat)
accuracy_score(ytrue,yhat)

1.0

In [12]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils # drawing utilities

def mediapipe_detection(image,model):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # color conversion
  image = cv2.flip(image, 1)
  image.flags.writeable = False
  results = model.process(image) # process image
  image.flags.writeable = True
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # color conversion
  return image,results

def draw_landmarks(image,result):
  if result.multi_hand_landmarks:
    for num,hand in enumerate(result.multi_hand_landmarks):
      mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(0, 112, 255), thickness=1, circle_radius=2),
                                mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=4),
                                )
def extract_keypoint(result):
  if result.multi_hand_landmarks:
    for hand_landmarks in result.multi_hand_landmarks:
      all_hand_pos = []
      for joint in mp_hands.HandLandmark:
        all_hand_pos.append(np.array([hand_landmarks.landmark[joint].x,hand_landmarks.landmark[joint].y,hand_landmarks.landmark[joint].z]))
  else:
    return np.zeros(21*3)
  return np.concatenate(all_hand_pos) # return 63 point of connection

colors = [(245,117,16), (117,245,16), (16,117,245),(44,217,235)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [15]:
# 1. New detection variables
sequence = []
word = []
prob = []
threshold = 0.8
detected_range = 5
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5,max_num_hands=1) as hands:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, hands)
        # Draw landmarks
        draw_landmarks(image, results)
        # 2. Prediction logic
        keypoints = extract_keypoint(results)
        sequence = keypoints
        res = model.predict(np.expand_dims(sequence, axis=0))[0]
        if res[np.argmax(res)] > threshold:
            if len(prob) == detected_range and all(x==prob[0] for x in prob):
                prob=[]
                if len(word)>0:
                    if alphabet[np.argmax(res)] != word[-1]:
                        word.append(alphabet[np.argmax(res)])
                else:
                    word.append(alphabet[np.argmax(res)])
            else:
                prob.append(alphabet[np.argmax(res)])
                if len(prob)>=detected_range:
                    prob = prob[-detected_range:]

        if len(word) > detected_range: 
                word = word[-detected_range:]
        
        # Viz probabilities
        # image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ''.join(word), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [18]:
cap.release()
cv2.destroyAllWindows()